## **Problem Statement**
How can we automatically generate accurate and concise summaries of informal, multi-turn chat conversations to help users quickly understand key discussion points without reading the entire dialogue?

## **Target Audience**
The target audience for this summarisation application includes individuals and organizations that frequently engage in or process informal, chat-based communication and require quick and coherent summaries for efficiency. For example; **journalists, customer support** & **call center teams**

## **Libraries**

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and

In [12]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=64c986cc6c406b656b3a0018f964ad9264a5f304f03d09c42ef8a5f70fd981f2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [2]:
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 16.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from evaluate import load as load_evaluator
from datasets import load_dataset

## **Dataset**

In [4]:
#Loading the SAMSum dataset which contains chat-style dialogues and summaries.
dataset = load_dataset("samsum")
dataset["train"] = dataset["train"].select(range(200))  # use only 200 samples for training
dataset["validation"] = dataset["validation"].select(range(50))  # use only 50 samples for validation

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

## **Preprocessing**

In [5]:
#Tokenize inputs and targets using BART tokenizer.
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

prefix = "summarize: "
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + dialogue for dialogue in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

In [6]:
#Tokenize the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## **Model Training**

In [7]:
#Set up model, data collator, training arguments and Trainer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

#Removed predict_with_generate as it's no longer a direct argument in TrainingArguments
training_args = TrainingArguments(
    output_dir="./bart-samsum-finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    #predict_with_generate=True, # This is handled during the prediction phase using the `trainer.predict()` function with the `generation_config` argument
    logging_dir="./logs",
    logging_steps=10,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-c9bd5458a45f>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.416700,1.436421
2,0.908900,1.399986


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=100, training_loss=1.2860404872894287, metrics={'train_runtime': 101.8578, 'train_samples_per_second': 3.927, 'train_steps_per_second': 0.982, 'total_flos': 232836765941760.0, 'train_loss': 1.2860404872894287, 'epoch': 2.0})

In [8]:
#Evaluating fine-tuned model on test data
results = []
for sample in dataset["test"]:
    input_text = prefix + sample['dialogue']
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    summary_ids = model.generate(**inputs, max_length=128, min_length=30, do_sample=False)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    results.append({
        'dialogue': sample['dialogue'],
        'reference_summary': sample['summary'],
        'generated_summary': generated_summary
    })


In [9]:
#Store results in a dataframe and export them to a CSV file
df = pd.DataFrame(results)
df.to_csv("bart_finetuned_samsum_results.csv", index=False)

## **Testing**

In [17]:
#Evaluate using ROUGE

print("Manual test examples:")
test_examples = [
    "John: Are you free tomorrow? Sarah: Yes, after 3 PM. John: Let's meet at 4 then!",
    "Alice: The report is due Friday. Bob: I thought it was Monday. Alice: No, it was moved up.",
    "Sam: I'm picking up groceries now. Need anything? Liz: Yes, some milk and eggs."
]

for text in test_examples:
    input_tensor = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    input_tensor = {key: val.to(model.device) for key, val in input_tensor.items()}
    summary_ids = model.generate(**input_tensor, max_length=60, min_length=20, do_sample=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(f"Input: {text}\nSummary: {summary}\n")

#Use ROUGE metrics to compare generated summaries to the reference ones
rouge = load_evaluator("rouge")

predictions = [x['generated_summary'] for x in results]
references = [x['reference_summary'] for x in results]

print("\nROUGE scores for Fine-tuned BART:")
rouge_output = rouge.compute(predictions=predictions, references=references)
print(rouge_output)

Manual test examples:
Input: John: Are you free tomorrow? Sarah: Yes, after 3 PM. John: Let's meet at 4 then!
Summary: Sarah is free tomorrow after 3 PM. John will meet with Sarah at 4 PM.

Input: Alice: The report is due Friday. Bob: I thought it was Monday. Alice: No, it was moved up.
Summary: Alice and Bob are due a report on Friday. Alice thought it was Monday.

Input: Sam: I'm picking up groceries now. Need anything? Liz: Yes, some milk and eggs.
Summary: Sam is picking up milk and eggs now. Liz will pick up the eggs and milk for Sam.


ROUGE scores for Fine-tuned BART:
{'rouge1': np.float64(0.44457673591948), 'rouge2': np.float64(0.20878116360141485), 'rougeL': np.float64(0.3477666440934998), 'rougeLsum': np.float64(0.34778876553748395)}
